# Trabalho Final (Algoritmo Branch and Bound)
Alunos: Adriel Ferreira Trajano e Kamily Assis de Oliveira

## Regra de ramificação

Nosso código ramifica em torno da variável $x_{j}$ cujo valor é fracionário e o mais próximo possível de 0,5. Em um filho, adiciona-se a restrição $x_{j}$ = 1; no outro, a restrição $x_{j}$ = 0.

## Estratégia de ramificação

A estratégia de ramificação do nosso código segue a abordagem padrão do branch and bound implementado com fila (queue).



In [1]:
from mip import Model, CBC ,MAXIMIZE,CONTINUOUS, xsum, OptimizationStatus

In [2]:
class Problem:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
        self.objective_coeffs = objective_coeffs #coef das funcoes objetivo.
        self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
        self.constraints_rhs = constraints_rhs # lados direitos das restricoes


In [3]:
class No:
    def __init__(self, model):
        self.model = model

    def add_constraint(self, constraint):
        self.model += constraint

In [4]:
def modelo(problema):

    qtd_restricoes = len(problema.constraints_coeffs)
    qtd_variaveis = len(problema.objective_coeffs)

    model = Model(sense=MAXIMIZE, solver_name=CBC)
    #lb e ub são os limites inferiores e superiores das VARIAVEIS e não do problema.
    x = [model.add_var(var_type=CONTINUOUS,name=f"x_{i}",lb=0,ub=1.0) for i in range(len(problema.constraints_coeffs))]


    model.objective = xsum(problema.objective_coeffs[i]*x[i] for i in range(qtd_variaveis))

    for i in range(qtd_restricoes):
        exp_restricao = xsum(problema.constraints_coeffs[i][j]*x[j] for j in range(qtd_variaveis))
        model += exp_restricao <= problema.constraints_rhs[i]

    return model

In [5]:
# identifica branching variable

def identify_branching_variable(node):
    variables = node.model.vars
    fractional_variables = [var for var in variables if var.x != int(var.x)]

    if not fractional_variables:
        return None

    # encontra a variável mais próxima de 0.5
    min_dif = min(abs(var.x - 0.5) for var in fractional_variables)
    branching_variable = [var for var in fractional_variables if abs(var.x - 0.5) == min_dif][0]

    return branching_variable

In [6]:
# funcao bound

class BranchAndBound():

    def __init__(self):
        self.primal = 0
        self.solution_vars = []

    def bound(self, node):
      node.model.optimize()

      if node.model.status == OptimizationStatus.INFEASIBLE:
        return "INVIAVEL",[], None

      if node.model.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return "NO SOLUTION",[], None

      if node.model.objective_value <= self.primal:
        return "PODADO",[], None

      if not identify_branching_variable(node):
        return "INTEGRALIDADE",node.model.vars,node.model.objective_value

      return "FRACIONARIO", node.model.vars, node.model.objective_value


    def branch_and_bound(self,node):
        queue = [node]

        while queue:
            status, var_solutions, objective = self.bound(queue[0])

            if status in ['INVIAVEL','NO SOLUTION','PODADO']:
                queue.pop(0)

            elif status == 'INTEGRALIDADE':
              if objective > self.primal:
                self.primal = objective
                self.solution_vars = var_solutions

            elif status == 'FRACIONARIO':
                no_explorado = queue.pop(0)
                branch_var = identify_branching_variable(no_explorado)

                # xj = 0
                cloned_model = no_explorado.model.copy()
                novo_no1 = No(cloned_model)
                new_constraint = branch_var == 0
                novo_no1.add_constraint(new_constraint)

                # xj = 1
                cloned_model = no_explorado.model.copy()
                novo_no2 = No(cloned_model)
                new_constraint = branch_var == 1
                novo_no2.add_constraint(new_constraint)

                queue.append(novo_no1)
                queue.append(novo_no2)


        return self.primal, self.solution_vars

In [7]:
def ler_dados(nome_arquivo):
    with open(nome_arquivo, 'r') as arq:
        linhas = arq.readlines()

    num_variaveis, num_restricoes = map(int, linhas[0].split())

    # coeficientes da funcao objetivo
    obj_coef = list(map(float, linhas[1].split()))

    restricoes = [] # coeficientes das restricoes
    rhs = [] # right hand side das restricoes

    for i in range(2, num_restricoes+2):
        restricoes.append(list(map(float, linhas[i].split()))[:-1])

    for i in range(2, num_restricoes+2):
        #rhs.append(list(map(float,linhas[i].split()[-2:-1])))
        rhs.append(float(linhas[i].split()[-1]))

    return Problem(obj_coef, restricoes, rhs)

In [8]:
main_path = '/Users/dorie/Downloads/UFPB/P4/PO/github/PESQUISA_OPERACIONAL/testes_utilizados'
arq1 =  main_path + '/teste1.txt'
arq2 = main_path + '/teste2.txt'
arq3 = main_path + '/teste3.txt'
arq4 = main_path + '/teste4.txt'

In [9]:
# testes

arq = {1:arq1,2:arq2,3:arq3,4:arq4}
for i in arq:
    
    print(f"Teste {i}:")

    problema = ler_dados(arq[i])
    model = modelo(problema)
    no = No(model)
    teste_bb = BranchAndBound()
    solucao_otm, vars = teste_bb.branch_and_bound(no)

    for var in vars:
        print(f"{var.name}: {var.x}")

    print("Solução Ótima: ", solucao_otm)
    print("\n")

Teste 1:
x_0: 0.0
x_1: 0.0
x_2: 0.0
x_3: 0.0
x_4: 1.0
x_5: 1.0
x_6: 0.0
x_7: 0.0
x_8: 0.0
x_9: 0.0
x_10: 0.0
Solução Ótima:  20.0


Teste 2:
x_0: 0.0
x_1: 0.0
x_2: 0.0
x_3: 0.0
x_4: 0.0
x_5: 1.0
x_6: 1.0
x_7: 0.0
x_8: 1.0
Solução Ótima:  24.0


Teste 3:
x_0: 0.0
x_1: 0.0
x_2: 1.0
x_3: 0.0
x_4: 0.0
x_5: 0.0
x_6: 1.0
x_7: 0.0
x_8: 0.0
x_9: 0.0
x_10: 0.0
x_11: 0.0
Solução Ótima:  19.0


Teste 4:
x_0: 0.0
x_1: 0.0
x_2: 1.0
x_3: 0.0
x_4: 0.0
x_5: 0.0
x_6: 0.0
x_7: 0.0
x_8: 0.0
Solução Ótima:  10.0


